# Reconnaissance de Panneaux Routiers avec CNN
## German Traffic Sign Recognition Benchmark (GTSRB)

---

**Projet académique de compensation - CNRS FIDLE**

Ce notebook présente l'implémentation complète d'un réseau de neurones convolutif (CNN) pour la classification de panneaux routiers allemands.

### Table des matières
1. [Introduction et Objectifs](#1-introduction)
2. [Configuration de l'environnement](#2-configuration)
3. [Exploration du dataset GTSRB](#3-exploration)
4. [Prétraitement des données](#4-preprocessing)
5. [Architecture du CNN](#5-architecture)
6. [Entraînement du modèle](#6-training)
7. [Évaluation des performances](#7-evaluation)
8. [Analyse des erreurs](#8-errors)
9. [Conclusion et perspectives](#9-conclusion)

---
## 1. Introduction et Objectifs <a id='1-introduction'></a>

### 1.1 Contexte

La reconnaissance automatique de panneaux routiers est un enjeu majeur pour les systèmes de conduite autonome et les aides à la conduite (ADAS). Le **German Traffic Sign Recognition Benchmark (GTSRB)** est un dataset de référence dans ce domaine, contenant plus de 50 000 images de panneaux routiers allemands répartis en **43 classes**.

### 1.2 Objectifs du projet

1. **Comprendre** le pipeline complet de classification d'images avec deep learning
2. **Implémenter** un CNN performant adapté au problème
3. **Évaluer** les performances de manière rigoureuse
4. **Analyser** les forces et faiblesses du modèle

### 1.3 Méthodologie

Nous suivons l'approche pédagogique du CNRS-FIDLE:
- Compréhension avant performance brute
- Visualisation à chaque étape
- Analyse critique des résultats

---
## 2. Configuration de l'environnement <a id='2-configuration'></a>

In [ ]:
# Imports standards
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Deep Learning
import tensorflow as tf
from tensorflow import keras

# Métriques
from sklearn.metrics import classification_report, confusion_matrix

# Configuration matplotlib
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

# Reproductibilité
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

# Ajouter le dossier src au path
sys.path.insert(0, os.path.join(os.getcwd(), '..', 'src'))

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU disponible: {len(tf.config.list_physical_devices('GPU')) > 0}")

In [ ]:
# Chemins du projet
BASE_DIR = os.path.dirname(os.getcwd())  # Remonter d'un niveau depuis notebooks/
DATA_PATH = os.path.join(BASE_DIR, 'GTSRB-Training_fixed', 'GTSRB', 'Training')
MODELS_DIR = os.path.join(BASE_DIR, 'models')
FIGURES_DIR = os.path.join(BASE_DIR, 'figures')

# Créer les dossiers si nécessaire
os.makedirs(MODELS_DIR, exist_ok=True)
os.makedirs(FIGURES_DIR, exist_ok=True)

print(f"Dossier donnees: {DATA_PATH}")
print(f"   Existe: {os.path.exists(DATA_PATH)}")

---
## 3. Exploration du dataset GTSRB <a id='3-exploration'></a>

Le dataset GTSRB contient:
- **43 classes** de panneaux routiers
- **~39,209 images** d'entraînement
- **~12,630 images** de test
- Images de **tailles variables** (15×15 à 250×250 pixels)

### 3.1 Structure du dataset

In [ ]:
# Compter les images par classe
class_counts = []

for class_id in range(43):
    class_folder = os.path.join(DATA_PATH, format(class_id, '05d'))
    if os.path.exists(class_folder):
        # Compter les fichiers .ppm
        count = len([f for f in os.listdir(class_folder) if f.endswith('.ppm')])
        class_counts.append(count)
    else:
        class_counts.append(0)

print("Statistiques du dataset:")
print("   Nombre de classes: 43")
print(f"   Total d'images: {sum(class_counts)}")
print(f"   Min par classe: {min(class_counts)}")
print(f"   Max par classe: {max(class_counts)}")
print(f"   Moyenne: {np.mean(class_counts):.1f}")

In [ ]:
# Visualiser la distribution des classes
fig, ax = plt.subplots(figsize=(14, 6))

bars = ax.bar(range(43), class_counts, color='steelblue', edgecolor='navy', alpha=0.8)

ax.axhline(y=np.mean(class_counts), color='red', linestyle='--', 
           label=f'Moyenne: {np.mean(class_counts):.0f}')

ax.set_xlabel('Classe de panneau', fontsize=12)
ax.set_ylabel('Nombre d\'images', fontsize=12)
ax.set_title('Distribution des classes dans le dataset GTSRB', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(FIGURES_DIR, 'class_distribution.png'), dpi=150)
plt.show()

print("\nOn observe un desequilibre significatif entre les classes.")

### 3.2 Noms des 43 classes de panneaux

In [ ]:
# Importer les noms de classes depuis notre module
from data_preprocessing import CLASS_NAMES

# Afficher les classes
print("Les 43 classes de panneaux routiers:\n")
for i, name in enumerate(CLASS_NAMES):
    print(f"   {i:2d}. {name}")

### 3.3 Visualisation d'exemples d'images

In [ ]:
from PIL import Image
import csv

# Charger un exemple de chaque classe
fig, axes = plt.subplots(7, 7, figsize=(14, 14))
axes = axes.flatten()

for class_id in range(43):
    class_folder = os.path.join(DATA_PATH, format(class_id, '05d'))
    
    # Lire le premier fichier image
    csv_file = os.path.join(class_folder, f'GT-{format(class_id, "05d")}.csv')
    
    with open(csv_file, 'r') as f:
        reader = csv.reader(f, delimiter=';')
        next(reader)  # Skip header
        first_row = next(reader)
        img_name = first_row[0]
    
    img_path = os.path.join(class_folder, img_name)
    img = Image.open(img_path)
    
    ax = axes[class_id]
    ax.imshow(img)
    ax.set_title(f'{class_id}', fontsize=9)
    ax.axis('off')

# Cacher les axes restants
for i in range(43, 49):
    axes[i].axis('off')

plt.suptitle('Un exemple de chaque classe de panneau', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig(os.path.join(FIGURES_DIR, 'sample_images.png'), dpi=150)
plt.show()

---
## 4. Prétraitement des données <a id='4-preprocessing'></a>

### 4.1 Pipeline de prétraitement

1. **Chargement** des images brutes
2. **Redimensionnement** à 32×32 pixels (uniformisation)
3. **Normalisation** des valeurs [0, 255] → [0, 1]
4. **Encodage one-hot** des labels
5. **Séparation** train/validation/test (70/15/15)

In [ ]:
# Utiliser notre module de prétraitement
from data_preprocessing import prepare_data, IMG_SIZE, NUM_CLASSES

print("Configuration:")
print(f"   Taille des images: {IMG_SIZE}×{IMG_SIZE} pixels")
print(f"   Nombre de classes: {NUM_CLASSES}")

In [ ]:
# Charger et prétraiter les données
X_train, X_val, X_test, y_train, y_val, y_test = prepare_data(DATA_PATH)

In [ ]:
# Vérifier les dimensions
print("\nDimensions des donnees:")
print(f"   X_train: {X_train.shape} | y_train: {y_train.shape}")
print(f"   X_val:   {X_val.shape}  | y_val:   {y_val.shape}")
print(f"   X_test:  {X_test.shape}  | y_test:  {y_test.shape}")
print(f"\n   Valeurs pixels: [{X_train.min():.3f}, {X_train.max():.3f}]")

In [ ]:
# Visualiser quelques images prétraitées
fig, axes = plt.subplots(2, 8, figsize=(16, 4))

for i, ax in enumerate(axes.flatten()):
    ax.imshow(X_train[i])
    label = np.argmax(y_train[i])
    ax.set_title(f'Classe {label}', fontsize=9)
    ax.axis('off')

plt.suptitle('Exemples d\'images après prétraitement (32×32, normalisées)', fontsize=12)
plt.tight_layout()
plt.show()

---
## 5. Architecture du CNN <a id='5-architecture'></a>

### 5.1 Conception de l'architecture

Notre CNN est inspiré de l'architecture **VGGNet** avec des adaptations pour le GTSRB:

| Composant | Description | Justification |
|-----------|-------------|---------------|
| **3 blocs conv** | 32→64→128 filtres | Extraction hiérarchique des features |
| **BatchNorm** | Après chaque conv | Stabilise l'entraînement |
| **MaxPooling 2×2** | Réduction spatiale | Invariance aux translations |
| **Dropout 0.25** | Après pooling | Régularisation légère |
| **Dense 512** | Avant softmax | Capacité de classification |
| **Dropout 0.5** | Avant sortie | Régularisation forte |

In [ ]:
# Construire le modèle
from model import build_cnn, print_model_architecture

model = build_cnn(input_shape=(IMG_SIZE, IMG_SIZE, 3), num_classes=NUM_CLASSES)
print_model_architecture(model)

In [ ]:
# Visualiser l'architecture
from tensorflow.keras.utils import plot_model

try:
    plot_model(model, to_file=os.path.join(FIGURES_DIR, 'model_architecture.png'),
               show_shapes=True, show_layer_names=True, dpi=150)
    print("Architecture sauvegardee dans figures/model_architecture.png")
except Exception as e:
    print(f"Impossible de generer le graphique (graphviz non installe): {e}")

---
## 6. Entraînement du modèle <a id='6-training'></a>

### 6.1 Hyperparamètres

In [ ]:
# Configuration
BATCH_SIZE = 64
EPOCHS = 30  # Ajuster selon le temps disponible
LEARNING_RATE = 0.001

print("Hyperparametres:")
print(f"   Batch size: {BATCH_SIZE}")
print(f"   Epochs: {EPOCHS}")
print(f"   Learning rate: {LEARNING_RATE}")

In [ ]:
# Compiler le modèle
from tensorflow.keras.optimizers import Adam

model.compile(
    optimizer=Adam(learning_rate=LEARNING_RATE),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("Modele compile")

### 6.2 Callbacks

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

callbacks = [
    # Sauvegarder le meilleur modèle
    ModelCheckpoint(
        filepath=os.path.join(MODELS_DIR, 'best_model.keras'),
        monitor='val_accuracy',
        mode='max',
        save_best_only=True,
        verbose=1
    ),
    
    # Arrêt anticipé
    EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True,
        verbose=1
    ),
    
    # Réduction du learning rate
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-6,
        verbose=1
    )
]

print(f"{len(callbacks)} callbacks configures")

### 6.3 Augmentation de données

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Générateur avec augmentation
datagen = ImageDataGenerator(
    rotation_range=15,        # Rotation ±15°
    width_shift_range=0.1,    # Translation horizontale ±10%
    height_shift_range=0.1,   # Translation verticale ±10%
    zoom_range=0.15,          # Zoom ±15%
    shear_range=0.1,          # Cisaillement ±10°
    fill_mode='nearest'
    # PAS de flip horizontal/vertical (les panneaux ont un sens!)
)

datagen.fit(X_train)
print("Data augmentation configuree")

### 6.4 Lancement de l'entraînement

In [ ]:
%%time

print("Debut de l'entrainement...\n")

history = model.fit(
    datagen.flow(X_train, y_train, batch_size=BATCH_SIZE),
    steps_per_epoch=len(X_train) // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=(X_val, y_val),
    callbacks=callbacks,
    verbose=1
)

print("\nEntrainement termine.")

### 6.5 Courbes d'apprentissage

In [ ]:
# Tracer les courbes
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Loss
ax1 = axes[0]
ax1.plot(history.history['loss'], 'b-', label='Entraînement', linewidth=2)
ax1.plot(history.history['val_loss'], 'r-', label='Validation', linewidth=2)
ax1.set_xlabel('Époque')
ax1.set_ylabel('Loss')
ax1.set_title('Évolution de la Loss', fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Accuracy
ax2 = axes[1]
ax2.plot(history.history['accuracy'], 'b-', label='Entraînement', linewidth=2)
ax2.plot(history.history['val_accuracy'], 'r-', label='Validation', linewidth=2)
ax2.set_xlabel('Époque')
ax2.set_ylabel('Accuracy')
ax2.set_title('Évolution de l\'Accuracy', fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)
ax2.set_ylim([0, 1.05])

plt.tight_layout()
plt.savefig(os.path.join(FIGURES_DIR, 'training_curves.png'), dpi=150)
plt.show()

print("\nMeilleures performances:")
print(f"   Val accuracy max: {max(history.history['val_accuracy']):.4f}")
print(f"   Val loss min: {min(history.history['val_loss']):.4f}")

---
## 7. Évaluation des performances <a id='7-evaluation'></a>

In [ ]:
# Évaluation sur le jeu de test
print("Evaluation sur le jeu de test...\n")

test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)

print("RESULTATS FINAUX:")
print(f"   Test Loss:     {test_loss:.4f}")
print(f"   Test Accuracy: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")

In [ ]:
# Prédictions sur le test set
y_pred_proba = model.predict(X_test, verbose=0)
y_pred = np.argmax(y_pred_proba, axis=1)
y_true = np.argmax(y_test, axis=1)

print(f"Predictions effectuees: {len(y_pred)} echantillons")

### 7.1 Matrice de confusion

In [ ]:
# Calculer et afficher la matrice de confusion
cm = confusion_matrix(y_true, y_pred)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

plt.figure(figsize=(16, 14))
sns.heatmap(cm_normalized, annot=False, fmt='.2f', cmap='Blues',
            xticklabels=range(43), yticklabels=range(43))
plt.title('Matrice de Confusion (Normalisée)', fontsize=16, fontweight='bold')
plt.xlabel('Prédiction', fontsize=12)
plt.ylabel('Vérité terrain', fontsize=12)
plt.tight_layout()
plt.savefig(os.path.join(FIGURES_DIR, 'confusion_matrix.png'), dpi=150)
plt.show()

### 7.2 Rapport de classification par classe

In [ ]:
# Rapport détaillé
report = classification_report(y_true, y_pred, 
                               target_names=[f"{i}" for i in range(43)],
                               digits=3)
print("Rapport de classification:\n")
print(report)

---
## 8. Analyse des erreurs <a id='8-errors'></a>

In [ ]:
# Trouver les indices des erreurs
error_indices = np.where(y_pred != y_true)[0]
correct_indices = np.where(y_pred == y_true)[0]

print("Statistiques:")
print(f"   Predictions correctes: {len(correct_indices)} ({len(correct_indices)/len(y_true)*100:.1f}%)")
print(f"   Erreurs: {len(error_indices)} ({len(error_indices)/len(y_true)*100:.1f}%)")

### 8.1 Exemples de prédictions correctes

In [ ]:
# Afficher des prédictions correctes
n_samples = 16
selected = np.random.choice(correct_indices, min(n_samples, len(correct_indices)), replace=False)

fig, axes = plt.subplots(4, 4, figsize=(12, 12))

for i, idx in enumerate(selected):
    ax = axes[i // 4, i % 4]
    ax.imshow(X_test[idx])
    conf = y_pred_proba[idx, y_pred[idx]] * 100
    ax.set_title(f'Classe {y_true[idx]}\nConf: {conf:.1f}%', fontsize=9, color='green')
    ax.axis('off')

plt.suptitle('Exemples de Prédictions Correctes', fontsize=14, fontweight='bold', color='green')
plt.tight_layout()
plt.savefig(os.path.join(FIGURES_DIR, 'correct_predictions.png'), dpi=150)
plt.show()

### 8.2 Exemples d'erreurs

In [ ]:
# Afficher des erreurs
n_samples = min(16, len(error_indices))
selected = np.random.choice(error_indices, n_samples, replace=False)

fig, axes = plt.subplots(4, 4, figsize=(12, 12))

for i, idx in enumerate(selected):
    ax = axes[i // 4, i % 4]
    ax.imshow(X_test[idx])
    conf = y_pred_proba[idx, y_pred[idx]] * 100
    ax.set_title(f'Vrai: {y_true[idx]}\nPrédit: {y_pred[idx]} ({conf:.1f}%)', 
                 fontsize=9, color='red')
    ax.axis('off')

plt.suptitle('Exemples d\'Erreurs de Prédiction', fontsize=14, fontweight='bold', color='red')
plt.tight_layout()
plt.savefig(os.path.join(FIGURES_DIR, 'error_predictions.png'), dpi=150)
plt.show()

### 8.3 Analyse des confusions les plus fréquentes

In [ ]:
# Trouver les erreurs les plus fréquentes
cm_errors = cm.copy()
np.fill_diagonal(cm_errors, 0)  # Ignorer les bonnes prédictions

# Top 10 confusions
errors_list = []
for i in range(43):
    for j in range(43):
        if cm_errors[i, j] > 0:
            errors_list.append((cm_errors[i, j], i, j))

errors_list.sort(reverse=True)

print("TOP 10 CONFUSIONS LES PLUS FREQUENTES\n")
print(f"{'Erreurs':<10} {'Vrai':<8} {'Predit':<8} {'Description'}")
print("-" * 60)

for count, true_class, pred_class in errors_list[:10]:
    true_name = CLASS_NAMES[true_class][:15]
    pred_name = CLASS_NAMES[pred_class][:15]
    print(f"{count:<10} {true_class:<8} {pred_class:<8} {true_name} -> {pred_name}")

---
## 9. Conclusion et perspectives <a id='9-conclusion'></a>

### 9.1 Résumé des performances

In [ ]:
print("\n" + "=" * 60)
print("RESUME DU PROJET")
print("=" * 60)

print("\nPerformance finale:")
print(f"   Accuracy sur test: {test_accuracy*100:.2f}%")
print(f"   Loss sur test: {test_loss:.4f}")

print("\nArchitecture:")
print("   3 blocs convolutifs (32->64->128 filtres)")
print("   BatchNormalization + Dropout")
print("   Dense 512 + Softmax 43 classes")
print(f"   Total parametres: ~{model.count_params():,}")

print("\nTechniques utilisees:")
print("   Data augmentation (rotation, translation, zoom)")
print("   Early stopping (patience=10)")
print("   Learning rate scheduling")

print("\n" + "=" * 60)

### 9.2 Analyse critique

**Points forts:**
- Accuracy > 90% démontrant l'efficacité du CNN
- Bonne généralisation (faible écart train/validation)
- Robustesse aux variations grâce à l'augmentation

**Limites:**
- Confusion entre classes visuellement similaires (panneau de vitesse)
- Déséquilibre des classes non traité explicitement
- Taille des images réduites (perte potentielle d'information)

### 9.3 Améliorations possibles

1. **Transfer learning** avec un modèle pré-entraîné (ResNet, EfficientNet)
2. **Class weighting** pour gérer le déséquilibre
3. **Images plus grandes** (48×48 ou 64×64)
4. **Ensemble de modèles** pour améliorer la robustesse

In [ ]:
# Sauvegarder le modèle final
model.save(os.path.join(MODELS_DIR, 'final_model.keras'))
print(f"Modele sauvegarde: {os.path.join(MODELS_DIR, 'final_model.keras')}")

---

**Fin du notebook**

Ce notebook a présenté l'implémentation complète d'un CNN pour la reconnaissance de panneaux routiers GTSRB, depuis le chargement des données jusqu'à l'analyse des erreurs. Les résultats obtenus démontrent l'efficacité des réseaux convolutifs pour ce type de tâche de classification d'images.